In [6]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BlueSkyHotelReviews").getOrCreate()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (139 kB/s)
Reading package lists... Done


In [ ]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.sql.types import StringType
from pyspark.ml.feature import StopWordsRemover

In [ ]:
# Create a DataFrame with hotel review dataset
dataframe = spark.read.csv("/content/hotel_reviews.csv", header=True)

In [ ]:
# Show DataFrame
dataframe.show()

+--------------------+------+--------------------+--------------------+
|          hotel_name|rating|               title|             content|
+--------------------+------+--------------------+--------------------+
|Rancho Valencia R...|     5|Best romantic vac...|Our experience at...|
|Rancho Valencia R...|     5|Sweet sweet serenity|Amazing place. Ev...|
|Rancho Valencia R...|     5|Amazing Property ...|We booked a 3 nig...|
| Aloft Arundel Mills|     2|Never again...bew...|Currently in bed ...|
| Aloft Arundel Mills|     5|ALWAYS GREAT STAY...|I live in Md and ...|
| Aloft Arundel Mills|     5|      Wonderful stay|I stayed here wit...|
| Aloft Arundel Mills|     5|     Worth the money|Beautiful rooms a...|
| Aloft Arundel Mills|     5|Great Hotel Exper...|We stayed here wh...|
| Aloft Arundel Mills|     5|Short stay for bu...|I travel a lot wi...|
|Hampton Inn Suite...|     5|Amazing employees...|In my line of wor...|
|Hampton Inn Suite...|     5|I loved our stay ...|The staff is v

In [ ]:
# Tokenize word
tokenizer = Tokenizer(inputCol="content", outputCol="tokenized_reviews")
tokenizer

Tokenizer_196fe70ce8bb

In [ ]:
# Transform and show DataFrame
tokenized = tokenizer.transform(dataframe)
tokenized.show(truncate=False)

+------------------------------------+------+---------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
def review_length(review):
  return len(review)


In [ ]:
review_length = udf(review_length, IntegerType())
review_length

<function __main__.review_length>

In [ ]:
# Select the needed columns and don't truncate results
print(tokenized)

tokenized.select("title", "content", "tokenized_reviews")\
    .withColumn("tokens", review_length(col("tokenized_reviews"))).show(truncate=False)


DataFrame[hotel_name: string, rating: string, title: string, content: string, tokenized_reviews: array<string>]
+---------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
tokenized.select("title", "content", "tokenized_reviews")\
    .withColumn("tokens", review_length(col("tokenized_reviews"))).agg({'tokens':'avg'}).show(truncate=False)

+-----------+
|avg(tokens)|
+-----------+
|59.0689    |
+-----------+



In [ ]:
dataframe.groupBy("hotel_name").agg({'rating':'avg'}).show()

+--------------------+------------------+
|          hotel_name|       avg(rating)|
+--------------------+------------------+
|Fairfield Inn Phi...|               4.2|
|    Beach Comber Inn|               5.0|
|Atlantis Oceanfro...|               4.5|
|Hampton Inn Suite...|               5.0|
|Extended Stay Ame...|               3.0|
|Comfort Suites Co...|               4.4|
|         Sound Board|             4.625|
|    Super 8 Anderson|               5.0|
|Chart House Suite...|               4.5|
|Budget Inn of Ame...|               4.0|
|Holiday Inn Expre...|               3.4|
|          Hotel Rose| 4.333333333333333|
|     Motel 6 Ardmore|               4.0|
|Magnuson Hotel Na...|               1.0|
|Hampton Inn Suite...|               4.0|
|Holiday Inn Expre...|2.3333333333333335|
|Alicia's Eagle Ro...|               3.0|
|Hampton Inn Roano...|               4.0|
|The Siena Hotel, ...|               5.0|
|Best Western Mt. ...|               5.0|
+--------------------+------------

In [ ]:
# Instantiate Remover
remover = StopWordsRemover(inputCol="tokenized_reviews", outputCol="stopword_filtered")

# Transform and show data
remover.transform(tokenized).show(truncate=False)

+------------------------------------+------+---------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Run the hashing term frequency
hashing = HashingTF(inputCol="tokenized_reviews", outputCol="hashedValues", numFeatures=pow(2,4))

# Transform into a DF
hashed_df = hashing.transform(tokenized)

In [ ]:
# Display new DataFrame
hashed_df.show(truncate=False)

+------------------------------------+------+---------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Fit the IDF on the data set 
idf = IDF(inputCol="hashedValues", outputCol="features")
idfModel = idf.fit(hashed_df)
rescaledData = idfModel.transform(hashed_df)

In [ ]:
# Display the DataFrame
rescaledData.select("content", "features").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|content                                                                                                                                                              